In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

# Define the state for user data
class UserState(TypedDict):
    is_premium: bool
    message: str

# Define nodes
def greet_user(state: UserState):
    state['message'] = "welcome to our service!"
    return state

def premium_greeting(state: UserState):
    state['message'] = "thank you for being a premium member!"
    return state

def regular_greeting(state: UserState):
    state['message'] = "consider upgrading to premium for more benefits."
    return state

# Define the decision node to choose the greeting based on user status
def check_subscription(state: UserState):
    if state['is_premium']:
        return 'premium_greeting'
    else:
        return 'regular_greeting'
    

# Build the state graph
graph_builder = StateGraph(UserState)
graph_builder.add_node("greet_user", greet_user)
graph_builder.add_node("premium_greeting", premium_greeting)
graph_builder.add_node("regular_greeting", regular_greeting)
graph_builder.add_node("check_subscription", check_subscription)


# Add edges to control flow
graph_builder.add_edge(START, "greet_user")
graph_builder.add_conditional_edges("greet_user", check_subscription)
graph_builder.add_edge( "premium_greeting", END)
graph_builder.add_edge( "regular_greeting", END)



/opt/anaconda3/envs/langchain_ai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# Compile and run for premium user
graph = graph_builder.compile()
result = graph.invoke({'is_premium': True, 'message': ''})
print(result)


{'is_premium': True, 'message': 'thank you for being a premium member!'}


In [3]:
# Compile and run for regular user
graph = graph_builder.compile()
result = graph.invoke({'is_premium': False, 'message': ''})
print(result)

{'is_premium': False, 'message': 'consider upgrading to premium for more benefits.'}


## New Conditional edges

### use preprocessing step and then conditional edges

In [1]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


#Define the state for user data
class GreetingState(TypedDict):
    greeting : str


#Define a preprocessing step to normalize the greeting
def normalize_greeting_node(state):

    #Transform the greeting to lowercase
    state['greeting'] = state['greeting'].lower()
    return state

#Define a node for the "Hi" greeting
def hi_greeting_node(state):
    state["greeting"] = "Hi there!" + state["greeting"]
    return state    

#Define a node for a standard greeting 
def regular_greeting_node(state):
    state["greeting"] = "Hello!" + state["greeting"]
    return state

#Define the conditional function to choose the greeting based on the normalized input
def conditional_greeting_node(state):
    if "hi" in state["greeting"]:
        return "hi_greeting"
    else:
        return "regular_greeting"


#Initialize the StateGraph 
builder = StateGraph(GreetingState)
builder.add_node("normalize_greeting", normalize_greeting_node)
builder.add_node("hi_greeting", hi_greeting_node)
builder.add_node("regular_greeting", regular_greeting_node)

#Add the START to normalization node  then conditionallly branch based on the transformed greeting
builder.add_edge(START, "normalize_greeting")
builder.add_conditional_edges("normalize_greeting", conditional_greeting_node, ["hi_greeting", "regular_greeting"])
builder.add_edge("hi_greeting", END)
builder.add_edge("regular_greeting", END)




/opt/anaconda3/envs/langchain_ai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
#Compile and run the graph with a greeting containing "hi"
graph = builder.compile()
result = graph.invoke({"greeting": "Hi, how are you?"})
print(result)

{'greeting': 'Hi there!hi, how are you?'}


In [3]:
#Compile and run the graph with a standard greeting
graph = builder.compile()
result = graph.invoke({"greeting": "Good morning!"})
print(result)

{'greeting': 'Hello!good morning!'}
